In [ ]:
!pip install scipy --upgrade

     |████████████████████████████████| 27.4MB 144kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
from functools import partial
from re import A

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma, softmax
from scipy.linalg import sqrtm
import scipy.stats

PI = 3.1415926535897932384626
E = 2.718281828459045
EPSILON = 0.000001

omega = lambda k: 2 * PI**((k-1)/2) / gamma((k-1)/2)
c = lambda k: 0.39894228040143267793**k

# utils
import numpy as np
from scipy.stats import multivariate_normal

def multidet(M):
  # Determinant of multidimensional matrix
  dm = np.zeros(M.shape[0])
  for _ in range(len(dm)):
    dm[_] = np.linalg.det(M[_])
  return dm[np.newaxis].T

# 90% sure this can be done using broadcasting:
def multimult(A, B):
  # "Broadcast" multiplication of matrices with vector
  dm = np.zeros(A.shape + B.shape)
  for _ in range(len(dm)):
    dm[_] = A[_] * B
  return dm

def gaussianExpectation(mu, cov, fn, n_samples=1000, presamples=None):
  if type(presamples) == type(None):
    samples = np.random.multivariate_normal(mu, cov, n_samples)
  else:
    samples = presamples
  return np.mean(np.apply_along_axis(fn, 1, samples), axis=0)

def mvNormpdf(x, mu, cov):
  k = mu.shape[0]
  return 1/((2*np.pi)**(k/2)*(np.linalg.det(cov)**0.5))* np.exp(-0.5 * ((x-mu).T @ (np.linalg.pinv(cov)) @ (x-mu)).sum())

def logRatio(mu1, mu2, sig1, sig2, w1, w2):
  return np.log(1 + (w2 * mvNormpdf(x, mu2, sig2))/(w1 * mvNormpdf(x, mu1, sig1)))

def pairwiseObjective(mu1, mu2, sig1, sig2, w1, w2):
  return -w1 * gaussianExpectation() - w2 * gaussianExpectation()

class Param:
  def __init__(self):
    self.m_prev = 0
    self.v_prev = 0
  
class VGMM:
    def __init__(self, k=1, d=1):
        """We have a few parameters:
        sigma_0 = base sigma
        lambda_i (i \in range(1, k+1)) = sigma multiple
        w_i (i \in range(1, k+1)) = weight
        mu_i = mean
        """

        self.sigma_0 = np.eye(d)
        self.lambdas = np.random.rand(k, 1)
        self.Ws = np.random.rand(k, 1)
        self.ws = softmax(self.Ws)
        self.mus = np.random.rand(k, d)

        self.k = k
        self.d = d

        self.eta = 0.03 # learning rate

        #adam
        self.beta_1 = 0.9
        self.beta_2 = 0.999
        self.epsilon = 10**(-8)
        self.m_prev = 0
        self.v_prev = 0
        self.adamparams = {}

        self.n_samples = 1000
        self.cgd = False

        self.whist = []
        self.lambdahist = [[] for i in range(k)]
        self.lambdaghist = [[] for i in range(k)]
        self.muhist = [[] for i in range(k)]
        self.h = []

    def get_grad(self, grad, params=None):
      return self.adam(grad, params=params)
    
    def adam(self, grad, params):
      if params['id'] not in self.adamparams:
        self.adamparams[params['id']] = Param()
      s = self.adamparams[params['id']]

      s.m_current = self.beta_1 * s.m_prev + (1 - self.beta_1) * grad
      s.v_current = self.beta_2 * s.v_prev + (1 - self.beta_2) * grad**2
      s.m_norm = s.m_current/(1 - self.beta_1**(params['t']+1))
      s.v_norm = s.v_current/(1 - self.beta_2**(params['t']+1))

      s.m_prev = s.m_current * 1.0
      s.v_prev = s.v_current * 1.0

      if params['id'] == 'lambda':
        self.h.append(self.eta * s.m_norm/((s.v_norm**0.5) + self.epsilon))
      return self.eta * s.m_norm/((s.v_norm**0.5) + self.epsilon)

    def setTarget(self, dist):
       # idea: compute sigma_0 based on ELBO
       dist1 = scipy.stats.norm(-1, 0.5).pdf
       dist2 = scipy.stats.norm(1, 2).pdf
       self.targetpdf = lambda x: 0.3 * dist1(x) + 0.7 * dist2(x)
       self.logp = lambda x: np.log(self.targetpdf(x)) if np.log(self.targetpdf(x)) > -100 else -100

    def coordinateDescent(self, rounds=500, entropyRounds=1, energyRounds=1):
        # TODO: allow "change < epsilon" flag
        maxrounds = rounds
        round = 0
        
        while True:
            self.round = round
            if round == maxrounds:
                break
            
            #print(self.mus.shape)
            if round % 10 == 0:
              print("round", round)
              pass
            for i in range(self.ws.shape[0]):
                self.entropyDescent(rounds=entropyRounds, param=("w", i))
                self.energyDescent(rounds=energyRounds, param=("w", i))
                self.ws = softmax(self.Ws)
            for i in range(self.lambdas.shape[0]):
                self.entropyDescent(rounds=entropyRounds, param=("lambda", i))
                self.energyDescent(rounds=energyRounds, param=("lambda", i))
                pass
            for i in range(self.mus.shape[0]):
                self.entropyDescent(rounds=entropyRounds, param=("mu", i))
                self.energyDescent(rounds=energyRounds, param=("mu", i))

            round += 1

    def entropyDescent(self, rounds=1, param=None):
        
        # maximize entropy wrt params 
        self.ws = softmax(self.Ws)
        for round in range(rounds):
            # update wrt individual entropies

            # det of arrays within array, see https://stackoverflow.com/questions/13393733/determinant-of-multidimensional-array for optimization options
            if round % 10 == 0:
              # print(round, multimult(self.lambdas, self.sigma_0))
              # print(round, self.mus)
              pass
            wGrads = -(np.log(self.ws)+1) + 1/2 * np.log(multidet(2*PI*E*multimult(self.lambdas**2, self.sigma_0))) * self.ws * (1 - self.ws)
            lambdaGrads = self.ws * self.d/(self.lambdas + EPSILON)

            var, ind = param
            # only update one variable at a time
            if var == "w" or not self.cgd:
                # mask = np.zeros(self.Ws.shape)
                # mask[ind] = np.ones(self.Ws[ind].shape)
                self.Ws += self.get_grad(wGrads, params={'t':self.round, 'id':'node-W'})
            
            if var == "lambda" or not self.cgd:
                # mask = np.zeros(self.lambdas.shape)
                # mask[ind] = np.ones(self.lambdas[ind].shape)
                self.lambdas += self.get_grad(lambdaGrads, params={'t':self.round, 'id':'node-lambdas'})

            # self.whist.append((len(self.whist), self.ws[0][0]))
            self.muhist[ind].append(1.0 * self.mus[ind])
            self.lambdahist[ind].append(1.0 * self.lambdas[ind])
            self.lambdaghist[ind].append(1.0 * lambdaGrads[ind])

            # update wrt pairwise entropies
            Q1SAMPLES = np.random.multivariate_normal(self.mus[ind], self.lambdas[ind]**2 * self.sigma_0, self.n_samples)
            self.Q1SAMPLES = Q1SAMPLES
            for j in range(self.k):
                if j != ind:
                    w1, w2 = self.ws[ind], self.ws[j]
                    sig1, sig2 = self.lambdas[ind]**2 * self.sigma_0, self.lambdas[j]**2 * self.sigma_0
                    sig1inv = np.linalg.inv(self.sigma_0 * self.lambdas[ind]**2)
                    sig2inv = np.linalg.inv(self.sigma_0 * self.lambdas[j]**2)
                    mu1, mu2 = self.mus[ind], self.mus[j]

                    if var == "w" or not self.cgd:
                      wfn1 = lambda x: np.log(1+(w2*mvNormpdf(x, mu2, sig2)/(w1 * mvNormpdf(x, mu1, sig1)))) + 1/(1+(w2*mvNormpdf(x, mu2, sig2)/(EPSILON + w1 * mvNormpdf(x, mu1, sig1)))) * (w2 * mvNormpdf(x, mu2, sig2)/(EPSILON + mvNormpdf(x, mu1, sig1))) * (-1/(EPSILON + w1))
                      wfn2 = lambda x: 1/(1 + (w1 * mvNormpdf(x, mu1, sig1))/(w2 * mvNormpdf(x, mu2, sig2))) * 1/(EPSILON + w2)
                      self.Ws[ind] += self.get_grad(-gaussianExpectation(None, None, wfn1, presamples=Q1SAMPLES) - w2 * gaussianExpectation(None, None, wfn2, presamples=Q1SAMPLES) * w1 * (1 - w1), params={'t':self.round, 'id':'pair-'+str(ind)+','+str(j)+'-W'})

                    if var == "mu" or not self.cgd:
                      omega1 = lambda x: 1/(1+(w2*mvNormpdf(x, mu2, sig2)/(EPSILON + w1 * mvNormpdf(x, mu1, sig1)))) * (w2 * mvNormpdf(x, mu2, sig2)/(EPSILON + mvNormpdf(x, mu1, sig1))) * (-1/(EPSILON + mvNormpdf(x, mu1, sig1)**2)) * mvNormpdf(x, mu1, sig1) * -sig1inv @ (x - mu1[np.newaxis])
                      mufn1 = lambda x: -sig1inv @ (x - mu1[np.newaxis]) * np.log(1+(w2*mvNormpdf(x, mu2, sig2)/(w1 * mvNormpdf(x, mu1, sig1)))) + omega1(x)
                      mufn2 = lambda x: mvNormpdf(x, mu2, sig2) * 1/(1 + (w1 * mvNormpdf(x, mu1, sig1))/(w2 * mvNormpdf(x, mu2, sig2))) * w1/(EPSILON + w2 * mvNormpdf(x, mu2, sig2)) * -sig1inv @ (x - mu1[np.newaxis])
                      self.mus[ind] += self.get_grad(np.squeeze((-w1 * gaussianExpectation(None, None, mufn1, presamples=Q1SAMPLES) - w2 * gaussianExpectation(None, None, mufn2, presamples=Q1SAMPLES)), axis=-1), params={'t':self.round, 'id':'pair-'+str(ind)+','+str(j)+'-mu'})
                      
                    if var == "lambda" or not self.cgd:
                      omega1 = lambda x: 1/(1+(w2*mvNormpdf(x, mu2, sig2)/(EPSILON + w1 * mvNormpdf(x, mu1, sig1)))) * (w2 * mvNormpdf(x, mu2, sig2)/(EPSILON + mvNormpdf(x, mu1, sig1))) * (-1/(EPSILON + mvNormpdf(x, mu1, sig1)**2)) * mvNormpdf(x, mu1, sig1) * (sig1inv - sig1inv @ (x - mu1[np.newaxis]) @ (x - mu1[np.newaxis]).T @ sig1inv) @ (self.lambdas[ind] * self.sigma_0)
                      lambdafn1 = lambda x: (sig1inv - sig1inv @ (x - mu1[np.newaxis]) @ (x - mu1[np.newaxis]).T @ sig1inv) @ (self.lambdas[ind] * self.sigma_0) * np.log(1+(w2*mvNormpdf(x, mu2, sig2)/(w1 * mvNormpdf(x, mu1, sig1)))) + omega1(x)
                      lambdafn2 = lambda x: mvNormpdf(x, mu2, sig2) * 1/(1 + (w1 * mvNormpdf(x, mu1, sig1))/(w2 * mvNormpdf(x, mu2, sig2))) * w1/(EPSILON + w2 * mvNormpdf(x, mu2, sig2)) * (sig1inv - sig1inv @ (x - mu1[np.newaxis]) @ (x - mu1[np.newaxis]).T @ sig1inv) @ (self.lambdas[ind] * self.sigma_0)
                      self.lambdas[ind] += self.get_grad(-w1 * gaussianExpectation(None, None, lambdafn1, presamples=Q1SAMPLES)[0][0] - w2 * gaussianExpectation(None, None, lambdafn2, presamples=Q1SAMPLES)[0][0], params={'t':self.round, 'id':'pair-'+str(ind)+','+str(j)+'-lambda'})

    def energyDescent(self, rounds=1, param="dummy variable"):
        # minimize energy $-\int q \log p$ wrt params
        self.ws = softmax(self.Ws)

        for round in range(rounds):
            for i in range(self.mus.shape[0]):
                mu = self.mus[i].T
                siginv = np.linalg.inv(self.sigma_0 * self.lambdas[i]**2)
                sig = self.sigma_0 * self.lambdas[i]** 2
                w = self.ws[i]

                mufn = lambda x: self.logp(x) * siginv @ (x - mu[np.newaxis])
                lambdafn = lambda x: self.logp(x) * -0.5 * (siginv - siginv @ (x - mu[np.newaxis]) @ (x - mu[np.newaxis]).T @ siginv) @ (2 * self.lambdas[i] * self.sigma_0)

                self.mus[i] -= self.get_grad(-1 * np.squeeze(self.ws[i] * gaussianExpectation(mu, sig, mufn), axis=-1), params={'t':self.round, 'id':'energy-mus'}) * 1.1
                self.lambdas[i] -= self.get_grad(-1 * self.ws[i] * gaussianExpectation(mu, sig, lambdafn)[0][0], params={'t':self.round, 'id':'energy-lambdas'}) # expectation should be diagonal matrix
                self.Ws -= self.get_grad(-1 * gaussianExpectation(mu, sig, self.logp) * w * (1 - w), params={'t':self.round, 'id':'energy-W'})

                # self.lambdahist.append((len(self.lambdahist), self.lambdas[0][0]))
                self.muhist[i].append(1.0 * self.mus[i])
                self.lambdahist[i].append(1.0 * self.lambdas[i])
    
    def gradHmu(self, ind):
      z = lambda i,j: mvNormpdf(self.mus[i], self.mus[j], (self.lambdas[i]**2+self.lambdas[j]**2)*self.sigma_0)

      grad = 0
      for i in range(self.k):
        if i != ind:
          S += - self.ws[i] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)]))) * self.ws[ind] * z(i, ind) * np.linalg.inv(self.sigma_0 * self.lambdas[ind]**2) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis])
        if i == ind:
          f = - self.ws[k] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)])))
          for j in range(self.k):
            if j != ind:
              S += f * self.ws[j] * z(i,j) * np.linalg.inv(self.sigma_0 * self.lambdas[i]**2) @ (self.mus[j][np.newaxis] - self.mus[i][np.newaxis])
            if j == ind:
              S += f * self.ws[j] * z(i,j) * -0.5 * np.log(np.linalg.det(2*self.lambdas[i]*self.sigma_0))
      return S
    
    def gradHsig(self, ind):
      z = lambda i,j: mvNormpdf(self.mus[i], self.mus[j], (self.lambdas[i]**2+self.lambdas[j]**2)*self.sigma_0)

      grad = 0
      for i in range(self.k):
        if i != ind:
          siginv = np.linalg.inv(self.sigma_0 * (self.lambdas[i]**2+self.lambdas[ind]**2))
          S += - self.ws[i] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)]))) * self.ws[ind] * z(i, ind) * 0.5 * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
        if i == ind:
          f = - self.ws[k] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)])))
          for j in range(self.k):
            if j != ind:
              siginv = np.linalg.inv(self.sigma_0 * (self.lambdas[i]**2+self.lambdas[j]**2))
              S += f * self.ws[j] * z(i, j)* 0.5 * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
            if j == ind:
              S += f * self.ws[j] * z(i, j) * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
      return S

    def gradHsig(self, ind):
      z = lambda i,j: mvNormpdf(self.mus[i], self.mus[j], (self.lambdas[i]**2+self.lambdas[j]**2)*self.sigma_0)

      grad = 0
      for i in range(self.k):
        if i != ind:
          siginv = np.linalg.inv(self.sigma_0 * (self.lambdas[i]**2+self.lambdas[ind]**2))
          S += - self.ws[i] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)]))) * self.ws[ind] * z(i, ind) * 0.5 * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
        if i == ind:
          f = - self.ws[k] * 1/(np.sum(np.array([self.ws[j] * z(i,j) for j in range(self.k)])))
          for j in range(self.k):
            if j != ind:
              siginv = np.linalg.inv(self.sigma_0 * (self.lambdas[i]**2+self.lambdas[j]**2))
              S += f * self.ws[j] * z(i, j)* 0.5 * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
            if j == ind:
              S += f * self.ws[j] * z(i, j) * (siginv - siginv @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]) @ (self.mus[i][np.newaxis] - self.mus[ind][np.newaxis]).T @ siginv)
      return S 
    
    # print results
    def printout(self, plot=False):
        print("weights", self.ws)
        print("mus", self.mus)
        print("sigma_0", self.sigma_0)
        print("lambdas", self.lambdas)

        if plot:
            if self.d > 2:
                raise ValueError("Dimensionality is too large for a plot of the resultant mixture.")

            clustersamples = np.random.multinomial(300, self.ws, size=1)

            data = np.array([])
            for i in range(len(clustersamples)):
                data = np.append(data, np.random.multivariate_normal(self.mus[i], self.lambdas[i] * self.sigma_0, clustersamples[i]))

            plt.hist(data, density=True)
            plt.show()
  

In [ ]:
vgmm = VGMM(k=1, d=1)

print(vgmm.ws)
print(vgmm.mus)
print(vgmm.lambdas)
print(vgmm.sigma_0)


[[1.]]
[[0.87288977]]
[[0.80488851]]
[[1.]]


In [ ]:
vgmm.setTarget(0) # hardcoded the target distribution, normally you'd pass a pdf


In [ ]:
vgmm.mus[0][0] = 2.2

In [ ]:
vgmm.coordinateDescent(rounds=1000)

round 0
round 10
round 20
round 30
round 40
round 50
round 60
round 70
round 80


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:207: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:207: RuntimeWarning: divide by zero encountered in true_divide


round 90
round 100
round 110
round 120
round 130
round 140
round 150
round 160
round 170
round 180
round 190
round 200
round 210
round 220
round 230
round 240
round 250
round 260
round 270
round 280
round 290
round 300
round 310
round 320
round 330
round 340
round 350
round 360
round 370
round 380
round 390
round 400
round 410
round 420
round 430
round 440
round 450
round 460
round 470
round 480
round 490
round 500
round 510
round 520
round 530
round 540
round 550
round 560
round 570
round 580
round 590
round 600
round 610
round 620
round 630
round 640
round 650
round 660
round 670


In [ ]:
plt.plot([i for i in vgmm.muhist[0]], label='line 1')
plt.plot([i for i in vgmm.lambdahist[1]], label='line 1')
#plt.plot([i for i in vgmm.lambdahist[0]])
#plt.plot([i for i in vgmm.lambdaghist[0]])
plt.show()

In [ ]:
vgmm.printout(plot=False)

In [ ]:
c = np.array(vgmm.mus)
rv = multivariate_normal(c, vgmm.lambdas[0]**2 * vgmm.sigma_0)
x = np.random.uniform(-5, 5, 5000)
y = rv.pdf(x)
z = vgmm.targetpdf(x)
plt.scatter(x, z, s=10) # target
plt.scatter(x, y, s=2) # vgmm
plt.show()

In [ ]:
vgmm.mus = np.array([vgmm.mus])
vgmm.mus

In [ ]:
vgmm.lambdas = np.array([[10.0]])

In [ ]:
fn = lambda x: x
gaussianExpectation(np.array([[2],[1]]), np.eye(2), fn, n_samples=50000)

In [ ]:
vgmm.v_current

In [ ]:
plt.plot([i for i in vgmm.lambdaghist[0]])
plt.plot([i for i in vgmm.lambdahist[0]])

plt.show()

In [ ]:
plt.plot([i[0][0] for i in vgmm.h])
plt.show()


In [ ]:
[i for i in vgmm.lambdahist[0]][0]